# Module 1 Practice: Server

In this practice, we will see how to create the server code of our Shiny apps. 

We know that a Shiny app has the following structure: 

In [ ]:
library(shiny)

# Define UI ----
ui <- fluidPage(
  
)

# Define server logic ----
server <- function(input, output) {
  
}

We know how to create a user interface from the UI practice notebook. Now, we focus on the server code. The server script has three locations to place the code. *How frequently a code runs* depends on those locations. 

In [ ]:
# server.R

# First place to put code 

server <- function(input, output) {
    
# Second place to put code 
    
    output$map <- renderPlot ({
        
        # Third place to put code: in this example, it's a plot output.

    })
        
    
  }


Code that is placed in the **first** place runs only **once** when the app is launched for the **first time** by any user. This is a good place to put the code that should run only once such as loading libraries, loading data sets, preprocessing data sets, etc. Any computationally heavy code should be put here so that the app does not slow down during interactive session. 

Code that is placed in the **second** place runs **once each time a new user visits the app**.  The unnamed function will run once and save a distinct set of reactive objects for each user. Any session or user related data should be handled here. 

Code that is placed in the **third** place runs **every time** the user changes the user interface widget that this particular output depends upon. We should not put unnecessary code here in order not to slow down the app. 

## Render and Output

UI code contains the **output** functions for text, data table, plot, and image. This functions depend on their corresponding **render** functions in the server code. For example, to have a text output, we use `textOutput()` function in ui.R that depends on the `renderPrint()` function in server.R, and similarly `plotOutput()` depends on `renderPlot()` and so on. 

We have seen an example of data table output in the lab notebook; the following shows how these functions are connected to each other: 

In [ ]:
# ui.R
library(shiny)
ui <- fluidPage(
  
sidebarLayout(
 sidebarPanel(),
    
 mainPanel(
 dataTableOutput("dataOutput") # this shows where the output goes fom the render function in server. 
 )
 
))
    
# server.R
server <- function(input, output) {
 
    output$dataOutput <- renderDataTable(mtcars) # using the same label "dataOutput", this tells how to render the output.  
}

As you can see, these two functions relate to each other by using the same given name/label **"dataOutput"**. You can give any variable name of your choosing in the Output functions of UI code; and that same variable name will be a field in the **`output` variable** in the server function.   

Let's see an example that also uses **`input`** variable to send user input back to the server code as well as plot functions. Pay attention to reactive expressions; they are used as functions, not as variables (e.g. formulaText**()**).

In [ ]:
# ui.R 
library(shiny)

# Define UI for miles per gallon app 
ui <- fluidPage(

  # App title 
  titlePanel("Miles Per Gallon"),

  sidebarLayout(

    # Sidebar panel for inputs
    sidebarPanel(

      # Input: Selector for variable to plot against mpg
      selectInput("variable", "Variable:",    # "variable" is the label for this widget
                  c("Cylinders" = "cyl",
                    "Transmission" = "am",
                    "Gears" = "gear")),

      # Input: Checkbox for whether outliers should be included 
      checkboxInput("outliers", "Show outliers", TRUE) # "outliers" is the label for this widget

    ),

    # Main panel for displaying outputs
    mainPanel(

      # Output: Formatted text for caption
      h3(textOutput("caption")), # "caption" is the label for this widget

      # Output: Plot of the requested variable against mpg
      plotOutput("mpgPlot")

    )
  )
)

In [ ]:
# server.R

library(shiny)
library(datasets)

# Data pre-processing
# Tweak the "am" variable to have nicer factor labels

# Since this doesn't rely on any user inputs, we can do this once at
# startup and then use the value throughout the lifetime of the app.
mpgData <- mtcars
mpgData$am <- factor(mpgData$am, labels = c("Automatic", "Manual"))


# Define server logic to plot various variables against mpg 

# input and output variables of this function provide communication with the UI above. 
server <- function(input, output) {

  # Compute the formula text 
  # This is in a reactive expression since it is shared by the
  # output$caption and output$mpgPlot functions
  formulaText <- reactive({
    paste("mpg ~", input$variable)
  })

  # Return the formula text to UI for printing as a caption 
  output$caption <- renderText({
    formulaText()
  })

  # Generate a plot of the requested variable against mpg
  # and exclude outliers if requested.
  output$mpgPlot <- renderPlot({
    boxplot(as.formula(formulaText()),
            data = mpgData,
            outline = input$outliers,
            col = "#75AADB", pch = 19)
  })

}

**Make sure to place the files under folder ```module1-practice3``` and deploy them. **

In [1]:
#DEPLOY TO SHINY SERVER
dir <- getwd() #This gets the current Working Directory
course <- "DATA-SCI-8654" #This is to specify the course path for the shiny server
folder <- "module1-practice3" #This specifies the folder name to copy

system(sprintf("/usr/local/bin/shiny_deploy %s %s %s", course, dir,folder), 
       intern = TRUE,
       ignore.stdout = FALSE, 
       ignore.stderr = FALSE,
       wait = TRUE, 
       input = NULL)

[1] "sending incremental file list"                                                                                   
[2] "module1-practice3/"                                                                                              
[3] "module1-practice3/server.R"                                                                                      
[4] "\r          1,145 100%    0.00kB/s    0:00:00  \r          1,145 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=1/3)"
[5] "module1-practice3/ui.R"                                                                                          
[6] "\r            912 100%  890.62kB/s    0:00:00  \r            912 100%  890.62kB/s    0:00:00 (xfr#2, to-chk=0/3)"
[7] ""                                                                                                                
[8] "sent 2,279 bytes  received 58 bytes  4,674.00 bytes/sec"                                                         
[9] "total size is 2,057  speedup is 0.88"

Let's go see how it runs. Enter the following URL to your browser's address bar: 
 
  shiny.dsa.missouri.edu/students/YOUR_USERNAME/DATA-SCI-8654/module1-practice3/
  

### Checking the error logs 

If you make a mistake, you'll see an error message in the Shiny app output when you try to run it; it will tell you to check the error logs. The error logs are located at **`/dsa/shared2/app_home/shiny/YOUR_USERNAME/ShinyApps/log/`** folder. 
  
You can run a terminal and navigate to that folder with `cd` command, and you will see a text file named something like **`module1-practice3-YOUR_USERNAME-DATE-ETC.log`**. You can look at that text file by running the `cat` command in the terminal window, and you'll see what went wrong with your app. 
  

Also, if you go to `/dsa/shared2/app_home/shiny/YOUR_USERNAME/ShinyApps/DATA-SCI-8654` folder, you'll see your deployed apps in several folders named same way as the app folders under your module folders. 
  
### Now your turn: 

Modify the above app to do the same using ggplot library. We will use the exact same ui.R as the interface does not depend on which library we use to plot data. We will modify the server.R in order to use geom_boxplot of ggplot. First we add library to the server.R: 

```
library(ggplot2)
```

Then we change the inside of ```renderPlot()``` function to call the ggplot: 

```
p <- ggplot(mpgData, aes(x= ... , y= ...)) + geom_boxplot()
print(p)
```

We use the ```print()``` function inside the ```renderPlot```; this is usually how the ggplot object is sent to the Shiny's function. 

Pay attention to the **```aes()```**; normally we put the attribute names there such as : 

```
aes(x=gear, y=mpg)
```

In Shiny, if we want these attribute names to be selected by user, we need to supply the string chosen by the user such as: 

```
aes(x=input$variable, y=mpg)
```

This way you can pass the attribute name in the variable, but this will produce an error; you have to use **aes_string()** instead: 

```
aes_string(x=input$variable, y=mpg)
```

This still gives error, because ```mpg``` is not a string, it's an attribute name. We fix it easily:

```
aes_string(x=input$variable, y="mpg")
```

Now it should work: 

```
p <- ggplot(mpgData, aes_string(x=input$variable, y="mpg")) + geom_boxplot()
print(p)
```

Let's see the whole server.R content : 




In [ ]:
# server.R

library(shiny)
library(ggplot2)
library(datasets)

# Data pre-processing
# Tweak the "am" variable to have nicer factor labels

# Since this doesn't rely on any user inputs, we can do this once at
# startup and then use the value throughout the lifetime of the app.
mpgData <- mtcars
mpgData$am <- factor(mpgData$am, labels = c("Automatic", "Manual"))


# Define server logic to plot various variables against mpg 

# input and output variables of this function provide communication with the UI above. 
server <- function(input, output) {

  # Compute the formula text 
  # This is in a reactive expression since it is shared by the
  # output$caption and output$mpgPlot functions
  formulaText <- reactive({
    paste("mpg ~", input$variable)
  })

  # Return the formula text to UI for printing as a caption 
  output$caption <- renderText({
    formulaText()
  })

  # Generate a plot of the requested variable against mpg
  # and exclude outliers if requested.
  output$mpgPlot <- renderPlot({
      
      p <- ggplot(mpgData, aes_string(x=input$variable, y="mpg")) + geom_boxplot()
      print(p)
      
  })

}

What do we do with the ```input$outliers``` ? There is a way incorporate it into the plot:

```
geom_boxplot(outlier.shape = NA)
```

We need another trick to use it with the ```input$outliers``` as we can't directly assign the variable to the shape: 

```
   p <- ggplot(mpgData, aes_string(x=input$variable, y="mpg")) 
   
   if (input$outliers) {   
     p <- p + geom_boxplot()
   } else {
     p <- p + geom_boxplot(outlier.shape = NA)
   }
   print(p)
```

This way we can add either one or the other boxplot geom. 

Write this code into server.R and copy the ui.R from above and place them under **```module1-practice4```** and deploy them. Then check the app to see if you got it right. 

**Do you see a problem ? **

We need to turn ```cyl``` and ```gear``` into factors, too. Add these two lines right after ```mpgData <- mtcars``` and deploy and run again:

```
mpgData$cyl <- factor(mpgData$cyl)
mpgData$gear <- factor(mpgData$gear)
```
